## Preprocessing

In [68]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [69]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cleaned_application_df = application_df.drop(['EIN', 'NAME', 'AFFILIATION','ORGANIZATION'], axis=1) 
     #['EIN', 'NAME', 'AFFILIATION','ORGANIZATION'], axis=1)

#Optimization #1 ... dropping context data that we don't have clarity on it's influence of the approval process at the Charity.

#The affiliation and/or organization columns could be skewing out model results.

cleaned_application_df.tail(10)

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34289,T3,C1000,Preservation,1,25000-99999,N,20197,1
34290,T4,C3000,Preservation,1,1-9999,N,5000,0
34291,T4,C3000,ProductDev,1,0,N,5000,0
34292,T4,C3000,ProductDev,1,0,N,5000,0
34293,T3,C1000,Preservation,1,0,N,5000,1
34294,T4,C1000,ProductDev,1,0,N,5000,0
34295,T4,C3000,ProductDev,1,0,N,5000,0
34296,T3,C2000,Preservation,1,0,N,5000,0
34297,T5,C3000,ProductDev,1,0,N,5000,1
34298,T3,C1000,Preservation,1,1M-5M,N,36500179,0


In [70]:
# Determine the number of unique values in each column.

app_df_unique_vals = cleaned_application_df.nunique()

print(app_df_unique_vals)

APPLICATION_TYPE            17
CLASSIFICATION              71
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [71]:
# Look at APPLICATION_TYPE value counts for binning

#cleaned_application_df
#["APPLICATION_TYPE"]

app_type_value_counts = cleaned_application_df["APPLICATION_TYPE"].value_counts()

app_type_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [72]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

cutoff_ref_optimized = 150  #1000 #5th time 1000 ... going with "much less"...    #4th time "10" ...  2nd 500 ... rather than the previous value of ...  100 ... 

comparative_values_listed = app_type_value_counts

application_types_to_replace = comparative_values_listed[comparative_values_listed <= cutoff_ref_optimized].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    cleaned_application_df['APPLICATION_TYPE'] = cleaned_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
cut_off_SanChk = cleaned_application_df['APPLICATION_TYPE'].value_counts()

cut_off_SanChk

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [73]:
# Look at CLASSIFICATION value counts for binning

classification_type_value_counts = cleaned_application_df['CLASSIFICATION'].value_counts()

classification_type_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [74]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_type_filtered_counts = classification_type_value_counts[classification_type_value_counts > 1]

# = [i for i in classification_type_value_counts if classification_type_value_counts > 1] #future fix ... using Chat GPT to help debug ... Same code as above is solution ...  
# = classification_type_value_counts.loc[lambda x: (x > 1)] ## Adds some "familiarity" ... to what lambda does as an alternate here ... 

print(classification_type_filtered_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [75]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

cutoff_ref_2 = 200   #6th ... try 2 ... is 200 ...  #5th time was at 1000 ... going with much less ...   #up to 4th ... 1000  #1st ... 1000

comparative_values_listed = classification_type_value_counts

classifications_to_replace= comparative_values_listed[comparative_values_listed <= cutoff_ref_2].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    cleaned_application_df['CLASSIFICATION'] = cleaned_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
cut_off_2_SanChk = cleaned_application_df['CLASSIFICATION'].value_counts()

cut_off_2_SanChk

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [76]:
cleaned_application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   CLASSIFICATION          34299 non-null  object
 2   USE_CASE                34299 non-null  object
 3   STATUS                  34299 non-null  int64 
 4   INCOME_AMT              34299 non-null  object
 5   SPECIAL_CONSIDERATIONS  34299 non-null  object
 6   ASK_AMT                 34299 non-null  int64 
 7   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 2.1+ MB


In [77]:
# Convert categorical data to numeric with `pd.get_dummies`

cat_col_names_fetched = [col_name for col_name, dtype in cleaned_application_df.dtypes.items() if dtype != 'int64']
#print(cat_col_names_fetched)

#[col_name for col_name in cleaned_application_df.columns if cleaned_application_df.columns.dtype == type('int64') ]   # != "int64" ]

#Chat GPT answer below:
# Use a list comprehension to get column names where the datatype is not "int64"
#non_int64_columns = [col for col, dtype in df.dtypes.items() if dtype != 'int64']

cat_transformed_application_df = pd.get_dummies(cleaned_application_df, columns=cat_col_names_fetched)

print(cat_col_names_fetched)
print("++++++")
#cleaned_application_df.head(4)

cat_transformed_application_df.head(4)

['APPLICATION_TYPE', 'CLASSIFICATION', 'USE_CASE', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
++++++


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0


In [78]:
# Split our preprocessed data into our features and target arrays

target_data_col = "IS_SUCCESSFUL" #"the rest" ... becomes the features ... 
#NOTE ... this is what "y_pred" ... eventually is giving us proposals (predictions) for ... 

X_feature_data = cat_transformed_application_df.drop(target_data_col, axis=1)
y_target_data = cat_transformed_application_df[target_data_col]

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X_feature_data, y_target_data, random_state=27)

print(y_test)
print("-----")
print(len(y_target_data))
print(len(y_test))
print(len(y_train))


27646    1
31610    0
32109    0
33466    1
30030    1
        ..
208      1
5249     1
20516    0
31793    0
22133    0
Name: IS_SUCCESSFUL, Length: 8575, dtype: int64
-----
34299
8575
25724


In [79]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#AoPers - Use Keras-Tuner to assist in Model Optimization

In [80]:
#Leveraging prior activity -> # Create a method that creates a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model_ktuner = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model_ktuner.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model_ktuner.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model_ktuner.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model_ktuner.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model_ktuner

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner_epochs = 25

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=tuner_epochs,
    hyperband_iterations=2    #Leaving at 2 ... 
    )

#hyperband_iterations via url => https://keras.io/api/keras_tuner/tuners/hyperband/#:~:text=hyperband_iterations%3A%20Integer%2C%20at%20least%201%2C%20the%20number%20of,a%20value%20as%20is%20within%20your%20resource%20budget.
##
# => hyperband_iterations: Integer, at least 1, the number of times to iterate over the full Hyperband algorithm. One iteration will run approximately max_epochs * (math.log(max_epochs, factor) ** 2) cumulative epochs across all trials. It is recommended to set this to as high a value as is within your resource budget. Defaults to 1.


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=tuner_epochs,validation_data=(X_test_scaled,y_test))

## Compile, Train and Evaluate the Model

In [63]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

X_features_datapoints_count = X_train_scaled.shape[1]    #X_feature_data[0])  #X_feature_data[1])  #X_train_scaled.shape[1]
nodes_layer1 = 7  #6th ... going up to 90  ...  #5th is at 7 ... 4th from 20 ... 1st from  80
nodes_layer2 = 12 #6th ... going up to 40  ...  #5th is at 12 ... 4th from 15 ... 1st from 30
dropout_rate1 = 0.5
nodes_layer3 = 25 #6th ... starting at 25
nodes_layer4 = 12 #6th ... starting at 12
node_outputlayer = 1
####
print(X_features_datapoints_count)  #6th at 36 ...  #5th at 30 ...  4th at 56 ... #4th back to 44 ...  #2nd .... Shape of inputs reduced from 44 to 33 ... 


36


In [64]:
#Define the model and apply the layers to check on it's structure ...
####

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=nodes_layer1, activation="relu",input_dim=X_features_datapoints_count))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=nodes_layer2, activation="relu"))

# 3rd hidden layer
nn_model2.add(tf.keras.layers.Dense(units=nodes_layer3, activation="relu"))

#Dropout Layer adding regularization step in process
nn_model2.add(tf.keras.layers.Dropout(dropout_rate1))

# 4th hidden layer
nn_model2.add(tf.keras.layers.Dense(units=nodes_layer4, activation="relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=node_outputlayer, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 7)                 259       
                                                                 
 dense_7 (Dense)             (None, 12)                96        
                                                                 
 dense_8 (Dense)             (None, 25)                325       
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense_9 (Dense)             (None, 12)                312       
                                                                 
 dense_10 (Dense)            (None, 1)                 13        
                                                                 
Total params: 1005 (3.93 KB)
Trainable params: 1005 (3

In [65]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [66]:
# Train the model

     #PersNote ... model appears to "cap out" ... at 50-65 epochs ... 
     #5th ... going with 32 for batch_size ... 55 for epochs ... 
     #will use 65 from 4th run onwards ... Also going from 32 to 16 for 4th run "try 2" ... 

fit_model2 = nn_model2.fit(X_train_scaled, y_train, epochs=55, batch_size=32)

#2nd ...
#Less epochs from 100 ... and added batch_size ... 
#Model started ... with much lower accuracy ... but it is increasing better as it goes ... 

#3rd ...
#ADDING BACK the other columns to see how that influenced the accuracy ... 

#4th ... 
#Will toggle epochs to higher ... at 175 ... + have made edits to the nodes + bins ... trying to let the model "see more data classes" ... 

#5th ... will let it see LESS data ... more MORE "other" ... by adjusting the bins ... 

#6th + ... just will comment in the print section for below ... 


fit_model2

Epoch 1/55



804/804 [==============================] - 3s 3ms/step - loss: 0.6742 - accuracy: 0.5779
Epoch 2/55

804/804 [==============================] - 3s 3ms/step - loss: 0.6483 - accuracy: 0.6196
Epoch 3/55

804/804 [==============================] - 2s 3ms/step - loss: 0.6423 - accuracy: 0.6225
Epoch 4/55

804/804 [==============================] - 2s 2ms/step - loss: 0.6398 - accuracy: 0.6205
Epoch 5/55

804/804 [==============================] - 2s 2ms/step - loss: 0.6368 - accuracy: 0.6261
Epoch 6/55

804/804 [==============================] - 2s 3ms/step - loss: 0.6352 - accuracy: 0.6296
Epoch 7/55

804/804 [==============================] - 2s 3ms/step - loss: 0.6357 - accuracy: 0.6285
Epoch 8/55

804/804 [==============================] - 2s 2ms/step - loss: 0.6346 - accuracy: 0.6279
Epoch 9/55

804/804 [==============================] - 2s 3ms/step - loss: 0.6345 - accuracy: 0.6255
Epoch 10/55

804/804 [==============================] - 2s 3ms/step - loss: 0.6341 - accuracy: 0.6262


In [67]:
# Evaluate the model using the test data
model2_loss, model2_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model2_loss}, Accuracy: {model2_accuracy}")

#First run ... is less accurate with less data inputs ...
#  resulted in .... 
#268/268 - 0s - loss: 0.6408 - accuracy: 0.6218 - 451ms/epoch - 2ms/step
#Loss: 0.640800416469574, Accuracy: 0.6218075752258301

#2nd run ... plan is ... lower epochs + add batch_size ... also change hidden layer nodes ... 
#  resulted in .... 
#268/268 - 1s - loss: 0.6349 - accuracy: 0.6218 - 525ms/epoch - 2ms/step
#Loss: 0.6349061727523804, Accuracy: 0.6218075752258301

#3rd run ... plan is ... keep epochs + batch_size ... add back in the columns I removed ... 
#  resulted in .... 
#268/268 - 1s - loss: 0.5567 - accuracy: 0.7321 - 635ms/epoch - 2ms/step
#Loss: 0.5567323565483093, Accuracy: 0.7321282625198364

#4th run ... plan is ... keep batch_size  ... increas epochs to 175 ... change bins ... to include more data ... 
#  resulted in ....  (Note this is for the "try 2" ... "try 1" ... never finished b/c it's epochs was way too high ... for my patience :-) ... )
#268/268 - 1s - loss: 0.5551 - accuracy: 0.7294 - 875ms/epoch - 3ms/step
#Loss: 0.555134117603302, Accuracy: 0.7294460535049438

#5th run ... plan is ... to include the suggestion of the call back to save weights ... 
#  resulted in .... (looks worst ... by early indications ... but adding in the dropout and such ... as "try 2" ... )
#268/268 - 1s - loss: 0.6377 - accuracy: 0.6180 - 563ms/epoch - 2ms/step
#Loss: 0.6377444267272949, Accuracy: 0.617959201335907

#6th .. try 2 ... Dropout didn't improve it ... 
#268/268 - 1s - loss: 0.6461 - accuracy: 0.6232 - 641ms/epoch - 2ms/step
#Loss: 0.6461184620857239, Accuracy: 0.6232069730758667


#7th ... see what keras-tuner provides me ... and NOTE that the 3rd run is the "best so far" ... 
#
#



268/268 - 1s - loss: 0.6461 - accuracy: 0.6232 - 641ms/epoch - 2ms/step
Loss: 0.6461184620857239, Accuracy: 0.6232069730758667


In [26]:
# Export our model to HDF5 file
nn_model2.save("AlphabetSoupCharity-optimized-nn_model_2.h5")

c:\Users\Aolda\anaconda3\envs\dev\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
#Trying out warning response on ".h5" being legacy ... 

nn_model2.save("AlphabetSoupCharity-optimized-nn_model_2.keras")